# Subspace Digital Twin, Initial Conditions Run

*Shawn Anderson, January 2024*

In this notebook, we examine medianl behavior over the first 90 days.

## Part 1. Dependences & Set-up

Autoreload modules while developing.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import numpy as np
import pandas as pd
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)

import hvplot.pandas
hvplot.extension('bokeh')

from bokeh.models import HoverTool
import holoviews as hv

from bokeh.palettes import Turbo256, Category20

from subspace_model.util import g


2024-01-18 14:27:27 - subspace-digital-twin - INFO
------------subspace-digital-twin------------


## Part 2. Load Simulation Data

Load the simulation results data.

In [2]:
sim_df = pd.read_pickle(
    "../data/simulations/reference_subsidy_sweep-2024-01-18_10-52-31.pkl.gz"
).drop(['timestep', 'simulation', 'subset', 'timestep_in_days', 'block_time_in_seconds', 'delta_days', 'delta_blocks'], axis=1)

In [3]:
sim_df.head(5)

,days_passed,blocks_passed,circulating_supply,user_supply,earned_supply,issued_supply,earned_minus_burned_supply,total_supply,sum_of_stocks,block_utilization,dsf_relative_disbursal_per_day,reward_issuance_balance,other_issuance_balance,operators_balance,nominators_balance,holders_balance,farmers_balance,staking_pool_balance,fund_balance,burnt_balance,nominator_pool_shares,operator_pool_shares,block_reward,history_size,space_pledged,allocated_tokens,buffer_size,reference_subsidy,average_base_fee,average_priority_fee,average_compute_weight_per_tx,average_transaction_size,transaction_count,average_compute_weight_per_bundle,average_bundle_size,bundle_count,compute_fee_volume,storage_fee_volume,rewards_to_nominators,run,average_compute_weight_per_budle,label,environmental_label,max_credit_supply
0,0,0.0,0.000000,0.000000,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.0,1.320000e+09,1.680000e+09,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,NaN,0,0,0.0,0.0,0.00000,0.0,0.0,0.0,256,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.0,1,NaN,constant-single-component,standard,3000000000
14,1,14400.0,13.698630,13.698630,0.0,1.680000e+09,0.0,1.680000e+09,3.000000e+09,0.5,0.0,1.320000e+09,1.680000e+09,0.000000,0.000000,0.684932,13.013699,0.0,0.000000,0.0,0.0,0.0,13.698630,2038498852864,1572944000000,0.0,60045568.0,13.69863,1.0,3.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.000000,0.000000,0.0,1,1.000000e+10,constant-single-component,standard,3000000000
28,2,28800.0,27.363014,27.397260,0.0,1.680000e+09,0.0,1.680000e+09,3.000000e+09,0.5,0.0,1.320000e+09,1.680000e+09,0.000000,0.000000,1.368151,25.994863,0.0,0.034247,0.0,0.0,0.0,13.698630,4076997705728,3145888000000,0.0,120091136.0,13.69863,1.0,3.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.342466,0.342466,0.0,1,1.000000e+10,constant-single-component,standard,3000000000
42,3,43200.0,41.201036,41.303690,0.0,1.680000e+09,0.0,1.680000e+09,3.000000e+09,0.5,0.0,1.320000e+09,1.680000e+09,0.000000,0.000000,2.060052,39.140985,0.0,0.102654,0.0,0.0,0.0,13.906430,6115764994048,4718832000000,0.0,45918976.0,13.69863,1.0,3.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.684075,0.684075,0.0,1,1.000000e+10,constant-single-component,standard,3000000000
56,4,57600.0,55.211744,55.417401,0.0,1.680000e+09,0.0,1.680000e+09,3.000000e+09,0.5,0.0,1.320000e+09,1.680000e+09,0.000745,0.000745,2.829882,52.380372,0.0,0.205657,0.0,0.0,0.0,14.113711,8154263846912,6291776000000,0.0,105964544.0,13.69863,1.0,3.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.955515,1.030026,0.0,1,1.000000e+10,constant-single-component,standard,3000000000


Simulation Runs.

In [4]:
sim_df.groupby(['run', 'label', 'environmental_label']).size().reset_index(name='Days').head()

,run,label,environmental_label,Days
0,1,constant-single-component,standard,362
1,2,hybrid-single-component,standard,362
2,3,hybrid-two-components,standard,362


### Coloring Metrics
Use a constant mapping from columns to colors

In [5]:
color_palette = Category20
# columns_to_color = sorted(list(set(sim_df.columns) - {'environmental_label', 'label', 'run', 'blocks_passed', 'days_passed'}))
columns_to_color = sim_df.columns
if color_palette == Turbo256:
    column_colors = dict(zip(columns_to_color, [color_palette[int(i)] for i in np.linspace(0,len(color_palette)-1, len(columns_to_color))]))

if color_palette == Category20:
    column_colors = {col: Category20[20][i%20] for i, col in enumerate(columns_to_color)}


sim_df.count().to_frame().T.hvplot.bar(y=columns_to_color, color=[column_colors[c] for c in columns_to_color], rot=90, width=1400, height=500, title='Column Color Map', fontscale=1.4, yaxis=None)

:Bars   [index,Variable]   (value)

In [6]:
def snake_to_title(s):
    """Utility function used for printing chart titles and labels as Title Case.
    Example:
    snake_to_caps('snake_case')
    >>> 'Snake Case'
    """
    
    return ' '.join(word.capitalize() for word in s.split('_'))

def fan_chart_quantile_median(df, column='circulating_supply', median_only=False):
    """Combine an area chart of min-max and a line chart of median for a series."""

    # min, max, median
    fan_df = df.groupby('days_passed')[column].agg(['min', 'max', 'median'])

    opts = dict(width=1200, height=500, title=f'{snake_to_title(column)} Fan Chart', ylabel=f'{column}_min_max_median')

    # Median curve
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Median', '@median{0,0.00}')])
    median_chart = fan_df.hvplot(x='days_passed', y='median', alpha=1, line_width=4, label=f'{snake_to_title(column)} Median', tools=[hover], color=column_colors[column]).opts(**opts)
    if median_only:
        return median_chart

    # min-max band
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Days Passed', '$x{0,0}')])
    bands_chart = fan_df.hvplot.area(x='days_passed', y='min', y2='max', legend='top_left', alpha=0.4, tools=[hover], ylim=(0,None), color=column_colors[column]).opts(**opts)

    # Composition
    chart = bands_chart * median_chart
    return chart


def fan_chart_quantile(df, column='circulating_supply', median_only=False):
    """Combine an area chart of min-max and a line chart of quantile for a series."""

    # 25%, 50%, 75%
    fan_df = df.groupby('days_passed')[column].quantile([0.25, 0.5, 0.75]).unstack().rename(columns={0.50:'median', 0.25:'0.25',0.75:'0.75'})

    # return fan_df

    opts = dict(width=1200, height=500, title=f'{snake_to_title(column)} Quantile Fan Chart', ylabel=f'{column}_quantile')

    # Quantile curve
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Median', '@median{0,0.00}')])
    quatile_chart = fan_df.hvplot(x='days_passed', y='median', alpha=1, line_width=4, label=f'{snake_to_title(column)} Quantile', tools=[hover], color=column_colors[column]).opts(**opts)
    if median_only:
        return quatile_chart

    # min-max band
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Days Passed', '$x{0,0}')])
    bands_chart = fan_df.hvplot.area(x='days_passed', y='0.25', y2='0.75', legend='top_left', alpha=0.4, tools=[hover], ylim=(0,None), color=column_colors[column]).opts(**opts)

    # Composition
    chart = bands_chart * quatile_chart
    return chart


In [7]:
system_balances = ['other_issuance_balance', 'reward_issuance_balance']
agent_balances = [
    'farmers_balance',
    'operators_balance',
    'user_supply',
    'nominators_balance',
]
agent_pool_balances = ['staking_pool_balance']
protocol_treasury_balances = ['fund_balance']
other_balances = list(set([c for c in sim_df.columns if 'balance' in c]) - set(system_balances + agent_balances + agent_pool_balances + protocol_treasury_balances) )
supply_columns = list({c for c in sim_df.columns if 'supply' in c} - {'max_credit_supply', 'issued_supply', 'total_supply'})
balance_columns = list(set([c for c in sim_df.columns if 'balance' in c]) - set(system_balances))

In [8]:
box_df = sim_df.set_index(['days_passed', 'label'])[balance_columns]
describe_df = box_df.describe().drop('count')
describe_df

,staking_pool_balance,farmers_balance,holders_balance,nominators_balance,burnt_balance,operators_balance,fund_balance
mean,190.198589,4507.675843,464.594906,141.361704,1.480259,35.712575,2920.368279
std,213.236135,3253.792197,338.013108,124.467882,1.997973,27.542633,2930.084254
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,18.635795,2124.719637,217.659677,39.568203,0.040763,15.255224,539.146440
50%,111.186020,3300.699256,337.944354,109.198740,0.542420,26.644121,2002.643044
75%,280.350808,6632.917132,685.431974,222.858478,2.197242,54.271763,4308.843456
max,879.672700,13262.761158,1375.020285,503.054718,8.534352,111.178932,12108.957618


In [9]:
describe_labels_df = box_df.groupby('label').apply(lambda label: label.describe().drop('count'))
describe_labels_df

staking_pool_balance  farmers_balance  \
label                                                                   
constant-single-component mean             93.204592      2061.619446   
                          std              90.335210       931.943584   
                          min               0.000000         0.000000   
                          25%              10.535022      1380.129874   
                          50%              64.002364      2298.738580   
                          75%             162.369727      2869.835339   
                          max             295.383173      3224.910930   
hybrid-single-component   mean            226.797358      4936.803416   
                          std             218.879934      2228.397666   
                          min               0.000000         0.000000   
                          25%              26.387691      3308.331313   
                          50%             156.657818      5503.976249   
                          75%             394.698848      6869.005674   
                          max             715.715309      7717.704462   
hybrid-two-components     mean            250.593817      6524.604668   
                          std             257.210453      3965.119622   
                          min               0.000000         0.000000   
                          25%              26.387691      3308.331313   
                          50%             156.657818      5503.976249   
                          75%             423.714764     10255.492511   
                          max             879.672700     13262.761158   

                                holders_balance  nominators_balance  \
label                                                                 
constant-single-component mean       211.632305           66.625361   
                          std         97.381032           45.191025   
                          min          0.000000            0.000000   
                          25%        140.220462           23.396775   
                          50%        236.370846           68.106842   
                          75%        296.149336          108.232046   
                          max        333.316885          138.156309   
hybrid-single-component   mean       509.350832          161.482523   
                          std        232.929006          108.783111   
                          min          0.000000            0.000000   
                          25%        338.743137           57.708598   
                          50%        568.560717          165.371541   
                          75%        711.442659          261.600313   
                          max        800.280261          333.254973   
hybrid-two-components     mean       672.801582          195.977228   
                          std        412.318439          153.910742   
                          min          0.000000            0.000000   
                          25%        338.743137           57.708598   
                          50%        568.560717          165.371541   
                          75%       1060.244421          323.179355   
                          max       1375.020285          503.054718   

                                burnt_balance  operators_balance  fund_balance  
label                                                                           
constant-single-component mean       0.739315          16.353293   1415.580784  
                          std        0.899679           8.572219   1177.686171  
                          min        0.000000           0.000000      0.000000  
                          25%        0.022443           9.576561    312.916120  
                          50%        0.305021          18.416910   1177.316229  
                          75%        1.241910          23.926661   2387.961308  
                          max        3.177245          27.352435   381

In [10]:
describe_difference_df = pd.DataFrame(describe_labels_df.values - pd.concat([describe_df for i in range(sim_df['label'].nunique())]).values, columns=describe_labels_df.columns, index=describe_labels_df.index)
df = describe_difference_df

def log_scale(val, max_abs_log):
    """ Apply logarithmic scaling to a value. """
    if val == 0:
        return 0
    else:
        return np.sign(val) * np.log(abs(val) + 1) / max_abs_log

def color_scale(val):
    max_abs_val = df.abs().max().max()
    max_abs_log = np.log(max_abs_val + 1)

    scaled_val = log_scale(val, max_abs_log)

    if scaled_val < 0:
        intensity = int(255 * (1 + scaled_val))  # More negative, more red
        return f'background-color: rgb(255, {intensity}, {intensity})'
    elif scaled_val > 0:
        intensity = int(255 * (1 - scaled_val))  # More positive, more green
        return f'background-color: rgb({intensity}, 255, {intensity})'
    else:
        return 'background-color: rgb(255, 255, 255)'

header_styles = [{
    'selector': f'th.col_heading.level0.col{i}',
    'props': [('background-color', column_colors.get(col))]
} for i, col in enumerate(df.columns)]

df.columns.name = 'balance'

describe_difference_df_styled = df.style.map(color_scale).set_table_styles(header_styles)
describe_difference_df_styled

In [11]:
box_df_melted = box_df.reset_index().drop('days_passed',axis=1).melt(id_vars=['label'])

violin_list = [label.hvplot.violin(y='value', by='variable', c='variable', legend='top_right', width=1200, height=500, title=f'SSC Balances {name}', cmap=column_colors, ylim=(0,box_df.max().max()*0.75)) for name, label in box_df_melted.groupby('label')]

# Combine plots into a single column layout
layout = hv.Layout(violin_list).cols(1)

layout

:Layout
   .Violin.I   :Violin   [variable]   (value)
   .Violin.II  :Violin   [variable]   (value)
   .Violin.III :Violin   [variable]   (value)

In [12]:
line_list = [hv.Overlay([fan_chart_quantile(label, column) for column in label.columns if 'balance' in column]).opts(title=f'SSC Balances {name}', legend_opts={'location':'top_left'}) for name, label in box_df.reset_index().groupby('label')]
layout = hv.Layout(line_list).cols(1)
layout

:Layout
   .Overlay.I   :Overlay
      .Area.I                              :Area   [days_passed]   (0.25,0.75)
      .Curve.Staking_Pool_Balance_Quantile :Curve   [days_passed]   (median)
      .Area.II                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Farmers_Balance_Quantile      :Curve   [days_passed]   (median)
      .Area.III                            :Area   [days_passed]   (0.25,0.75)
      .Curve.Holders_Balance_Quantile      :Curve   [days_passed]   (median)
      .Area.IV                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Nominators_Balance_Quantile   :Curve   [days_passed]   (median)
      .Area.V                              :Area   [days_passed]   (0.25,0.75)
      .Curve.Burnt_Balance_Quantile        :Curve   [days_passed]   (median)
      .Area.VI                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Operators_Balance_Quantile    :Curve   [days_passed]   (median)
      .Area.VII                            :Area   [days_passed]   (0.25,0.75)
      .Curve.Fund_Balance_Quantile         :Curve   [days_passed]   (median)
   .Overlay.II  :Overlay
      .Area.I                              :Area   [days_passed]   (0.25,0.75)
      .Curve.Staking_Pool_Balance_Quantile :Curve   [days_passed]   (median)
      .Area.II                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Farmers_Balance_Quantile      :Curve   [days_passed]   (median)
      .Area.III                            :Area   [days_passed]   (0.25,0.75)
      .Curve.Holders_Balance_Quantile      :Curve   [days_passed]   (median)
      .Area.IV                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Nominators_Balance_Quantile   :Curve   [days_passed]   (median)
      .Area.V                              :Area   [days_passed]   (0.25,0.75)
      .Curve.Burnt_Balance_Quantile        :Curve   [days_passed]   (median)
      .Area.VI                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Operators_Balance_Quantile    :Curve   [days_passed]   (median)
      .Area.VII                            :Area   [days_passed]   (0.25,0.75)
      .Curve.Fund_Balance_Quantile         :Curve   [days_passed]   (median)
   .Overlay.III :Overlay
      .Area.I                              :Area   [days_passed]   (0.25,0.75)
      .Curve.Staking_Pool_Balance_Quantile :Curve   [days_passed]   (median)
      .Area.II                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Farmers_Balance_Quantile      :Curve   [days_passed]   (median)
      .Area.III                            :Area   [days_passed]   (0.25,0.75)
      .Curve.Holders_Balance_Quantile      :Curve   [days_passed]   (median)
      .Area.IV                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Nominators_Balance_Quantile   :Curve   [days_passed]   (median)
      .Area.V                              :Area   [days_passed]   (0.25,0.75)
      .Curve.Burnt_Balance_Quantile        :Curve   [days_passed]   (median)
      .Area.VI                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Operators_Balance_Quantile    :Curve   [days_passed]   (median)
      .Area.VII                            :Area   [days_passed]   (0.25,0.75)
      .Curve.Fund_Balance_Quantile         :Curve   [days_passed]   (median)

In [13]:
violin_list = [variable.hvplot.violin(y='value', by='label', color=column_colors[name], width=1200, height=500, title=f'SSC Balances {name}', ylim=(0,variable.max()['value'].max())) for name, variable in box_df_melted.groupby('variable')]

layout = hv.Layout(violin_list).cols(1).opts(shared_axes=False)

layout

:Layout
   .Violin.I   :Violin   [label]   (value)
   .Violin.II  :Violin   [label]   (value)
   .Violin.III :Violin   [label]   (value)
   .Violin.IV  :Violin   [label]   (value)
   .Violin.V   :Violin   [label]   (value)
   .Violin.VI  :Violin   [label]   (value)
   .Violin.VII :Violin   [label]   (value)

In [36]:
line_list = [variable.hvplot.line(x='days_passed', by='label', y='value', title=name, legend='top_left', line_width=3).opts(legend_opts={'background_fill_color': column_colors[name], 'background_fill_alpha': 0.2}) for name, variable in box_df.reset_index().melt(id_vars=['label', 'days_passed']).groupby('variable')]

layout = hv.Layout(line_list).cols(2).opts(shared_axes=False)
layout

:Layout
   .NdOverlay.I   :NdOverlay   [label]
      :Curve   [days_passed]   (value)
   .NdOverlay.II  :NdOverlay   [label]
      :Curve   [days_passed]   (value)
   .NdOverlay.III :NdOverlay   [label]
      :Curve   [days_passed]   (value)
   .NdOverlay.IV  :NdOverlay   [label]
      :Curve   [days_passed]   (value)
   .NdOverlay.V   :NdOverlay   [label]
      :Curve   [days_passed]   (value)
   .NdOverlay.VI  :NdOverlay   [label]
      :Curve   [days_passed]   (value)
   .NdOverlay.VII :NdOverlay   [label]
      :Curve   [days_passed]   (value)